In [1]:
%pip install -qU tensorflow-cpu tf-keras sentence-transformers "protobuf~=3.20.2"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.manifold import TSNE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sentence_transformers import SentenceTransformer

2025-05-02 22:25:59.154053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746217559.166814  106344 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746217559.170276  106344 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746217559.181052  106344 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746217559.181067  106344 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746217559.181068  106344 computation_placer.cc:177] computation placer alr

based on : https://pypi.org/project/sentence-transformers/

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
df = pd.read_csv('data/test_labeled.csv') 

sentences = pd.DataFrame(df)

sentences = sentences.drop_duplicates()

In [5]:
def preprocess_text(text):
    text = text.lower() 
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text) 
    text = re.sub(r'[^a-zA-Z\s]', '', text)  
    return text


sentences['cleaned_premise'] = sentences['premise'].apply(preprocess_text)
sentences['cleaned_hypothesis'] = sentences['hypothesis'].apply(preprocess_text)
print(sentences['cleaned_premise'])

0                                                        
1                                                        
2       et cela est en grande partie d au fait que les...
3                                                  imaamp
4                                                        
                              ...                        
5190                                                     
5191    the  rock  has a soft texture and can be bough...
5192                                                     
5193    isnt it i can remember ive only been here eigh...
5194    in hong kong you can have a plate or even a wh...
Name: cleaned_premise, Length: 5195, dtype: object


In [6]:
embeddings_premises = model.encode(sentences['cleaned_premise'].tolist(), show_progress_bar=True)
embeddings_hypotheses = model.encode(sentences['cleaned_hypothesis'].tolist(), show_progress_bar=True)


Batches:   0%|          | 0/163 [00:00<?, ?it/s]

Batches:   0%|          | 0/163 [00:00<?, ?it/s]

In [7]:
print(embeddings_premises.shape)
print(embeddings_hypotheses.shape)

(5195, 384)
(5195, 384)


In [8]:
X = np.hstack((np.array(embeddings_premises) ,np.array(embeddings_hypotheses)))

y = sentences['label'].values

print(X.shape)
print(y.shape)

(5195, 768)
(5195,)


In [9]:
indices = np.arange(len(X))

X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y, indices, test_size=0.2, random_state=42
)

clf =  RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4156, 768)
(4156,)
(1039, 768)
(1039,)


In [11]:
y_pred = clf.predict(X_test)

premise_predicted = sentences.iloc[test_indices]['premise']
label_real = sentences.iloc[test_indices]['label']
hypothesis_predicted = sentences.iloc[test_indices]['hypothesis']

for premise,hypothesis, label_real, label_pred in zip(premise_predicted.head(30),hypothesis_predicted.head(30),label_real.head(30), y_pred[:30]):
    print(f"Premise: {premise}\nHypothesis: {hypothesis}\nReal label: {label_real}\nPredicted label: {label_pred}\nProximity: {clf.predict_proba([X_test[0]])}\n")   
    
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Premise: LSC set a deadline of October 1, 1998, for submission of state planning reports.
Hypothesis: LSC has a deadline of October 1,1998 to submit state planning reports.
Real label: 0
Predicted label: 2
Proximity: [[0.285      0.23833333 0.47666667]]

Premise: Turizm ofisleri L'Estrie bölgesini yeniden adlandırmaya çalıştılar ancak en militan Quebecli bile Cantons de l'Est'in doğrudan, daha yaklaşık çevirisini tercih eder.
Hypothesis: Turizmciler eskisi kulağa çirkin geldiğinden, bölgenin daha iyi bir isme ihtiyacı olduğunu düşünüyor.
Real label: 1
Predicted label: 0
Proximity: [[0.285      0.23833333 0.47666667]]

Premise: Given the limits on the WTO's jurisdiction, it was probably unreasonable of Kodak to expect a real victory.
Hypothesis: Kodak was naive and is still just a baby of a company.
Real label: 1
Predicted label: 2
Proximity: [[0.285      0.23833333 0.47666667]]

Premise: uh-huh yeah yeah they're good
Hypothesis: They are all right but not great.
Real label: 1
Predicted

In [12]:
num_classes = 3
input_dim = X.shape[1]

In [13]:
model = Sequential([
    Dense(512, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')  # Pour classification multi-classes
])

/home/adrien/Documents/TPS/2A/NLI/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-02 22:27:16.459102: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3569 - loss: 1.1002 - val_accuracy: 0.3918 - val_loss: 1.0789
Epoch 2/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4111 - loss: 1.0546 - val_accuracy: 0.3894 - val_loss: 1.0693
Epoch 3/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5005 - loss: 0.9740 - val_accuracy: 0.4147 - val_loss: 1.0830
Epoch 4/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5769 - loss: 0.8610 - val_accuracy: 0.4159 - val_loss: 1.1622
Epoch 5/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6720 - loss: 0.7238 - val_accuracy: 0.3690 - val_loss: 1.3066
Epoch 6/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7363 - loss: 0.5727 - val_accuracy: 0.3990 - val_loss: 1.4568
Epoch 7/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7755 - loss: 0.4898 - val_accuracy: 0.4038 - val_loss: 1.6514
Epoch 8/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8023 - loss: 0.4137 - val_accuracy: 0.

In [15]:
y_pred_classes = model.predict(X_test)
print(y_pred_classes.shape)
y_pred = np.argmax(y_pred_classes, axis=1)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
(1039, 3)


In [16]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 36.38%
